# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qaqortoq,60.7167,-46.0333,7.55,79,47,0.63,GL,1721353748
1,1,baiyin,36.5583,104.2081,23.05,65,98,1.74,CN,1721353748
2,2,bethel,41.3712,-73.4140,23.30,79,0,0.00,US,1721353748
3,3,vanino,49.0869,140.2543,15.81,99,100,1.84,RU,1721353748
4,4,puerto natales,-51.7236,-72.4875,4.25,81,40,4.27,CL,1721353749


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
##observing humidity 
plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True, 
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500, 
    color = 'City',
    alpha = 0.5,
    s= 'Humidity',
    scale = 0.8
)

# Display the map
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city = city_data_df.loc[(city_data_df["Max Temp"]<=20)\
                            & (city_data_df["Max Temp"]>=10) \
                            & (city_data_df["Cloudiness"]<=50)\
                            & (city_data_df["Cloudiness"]>=0)\
                            & (city_data_df["Country"]!= "US")\
                            & (city_data_df["Humidity"]<50),:
                            ]


ideal_city.dropna()

# Display sample data
ideal_city.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,alice springs,-23.7000,133.8833,18.75,26,0,5.14,AU,1721353819
75,75,kalabo,-14.9700,22.6814,11.61,21,0,1.08,ZM,1721353823
120,120,luau,-10.7073,22.2247,14.15,32,5,2.06,AO,1721353830
228,228,mount isa,-20.7333,139.5000,18.87,26,6,5.14,AU,1721353846
275,275,ghanzi,-21.5667,21.7833,13.07,18,0,4.04,BW,1721353852


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] =" "
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,qaqortoq,GL,60.7167,-46.0333,79,
1,baiyin,CN,36.5583,104.2081,65,
2,bethel,US,41.3712,-73.4140,79,
3,vanino,RU,49.0869,140.2543,99,
4,puerto natales,CL,-51.7236,-72.4875,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accomodation.hotel",\
    "filter":"",\
    "bias":"",\
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
qaqortoq - nearest hotel: No hotel found
baiyin - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
vanino - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
starobil's'k - nearest hotel: No hotel found
yatou - nearest hotel: No hotel found
wewak - nearest hotel: No hotel found
dudinka - nearest hotel: No hotel found
nemuro - nearest hotel: No hotel found
cidreira - nearest hotel: No hotel found
veinticinco de mayo - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
ponta delgada - nearest hotel: No hotel found
zouerate - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
thompson - nearest hotel: No hotel found
el granada - nearest hotel: No hote

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True, 
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500, 
    color = 'city',
    alpha = 0.5,
    s= 'Humidity',
    scale = 0.8,
    hover_cols= ["Hotel Name", "Country"]
)
# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)